# Mobile Phone Analysis

Analysis of phone characteristics and popularity of different phone models collected on 14th of March

In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns

In [2]:
df = pd.read_csv('../clean_phones/relevant_features.csv')
df = df[df['type'] == 'phone']
df.head()

,brand,model,photo_link,phone_link,popularity_become_fan,popularity_views,popularity_views_today,price,eSIM,announce_year,...,radio,usb_type,usb_version,biometric_auth,has_black_color,foldable,battery_type,battery_capacity,type,5g
0,alcatel,1B (2022),https://fdn2.gsmarena.com/vv/bigpic/alcatel-1b...,https://www.gsmarena.com/alcatel_1b_(2022)-117...,18,252662,0.3,100.0,False,2022.0,...,True,microUSB,2.0,False,True,False,Li-Ion,3000.0,phone,False
1,alcatel,1L Pro (2021),https://fdn2.gsmarena.com/vv/bigpic/alcatel-1l...,https://www.gsmarena.com/alcatel_1l_pro_(2021)...,14,207311,0.1,110.0,False,2021.0,...,True,microUSB,2.0,True,False,False,Li-Ion,3000.0,phone,False
2,alcatel,1 (2021),https://fdn2.gsmarena.com/vv/bigpic/alcatel-1-...,https://www.gsmarena.com/alcatel_1_(2021)-1098...,19,238822,0.1,60.0,False,2021.0,...,True,microUSB,2.0,False,True,False,Li-Ion,2000.0,phone,False
3,alcatel,3L (2021),https://fdn2.gsmarena.com/vv/bigpic/alcatel-a3...,https://www.gsmarena.com/alcatel_3l_(2021)-106...,20,218264,0.0,330.0,False,2021.0,...,True,microUSB,2.0,True,True,False,Li-Po,4000.0,phone,False
4,alcatel,1S (2021),https://fdn2.gsmarena.com/vv/bigpic/alcatel-1s...,https://www.gsmarena.com/alcatel_1s_(2021)-106...,18,259768,0.1,130.0,False,2021.0,...,True,microUSB,2.0,True,True,False,Li-Po,4000.0,phone,False


I will select top 10 phones by average likes on each phone in last 5 years

In [3]:
top_brands = df.groupby('brand')[['popularity_become_fan']].mean('popularity_become_fan').sort_values('popularity_become_fan', ascending=False).reset_index().iloc[:10]['brand']
df = df[df['brand'].isin(top_brands)]

In [4]:
pdf = df.groupby('brand').size().reset_index(name="counts")
px.bar(pdf.sort_values('counts', ascending=False), x='brand', y='counts')

The plot shows that Xiaomi in the years 2020-2025 has released the most amount of smartphones, followed by oppo and realme.

This might mean that they try to target all market segments with their phone models. This will be investigated in consequent graphs.

The most frequent price segment for phones in recent years is budget-friendly, from 75 up to 225 euros. This might imply that there is a lot of competition in budget phonne segment, with many manufacturers offering many different phones in this price range.

A long tail with low counts imply that there are a few of premium flagship phones.

In [5]:
pdf = df[['popularity_become_fan']]
px.histogram(pdf, x='popularity_become_fan', nbins=100)

Most phones recieve from 20-40 likes

In [6]:
pdf = df[['price', 'brand']]
px.box(pdf.sort_values('price'), x='price', y='brand')

The box plot shows the distribution of price between top 10 brands.

In [7]:
pdf = df
# sns.histplot(pdf, x='price', hue='brand', bins=[0,200,780,1500,5000])
bins = [0, 225, 780, 1500, pdf['price'].max()]  # Define the edges of the bins
labels = ['0-225', '225-780', '780-1500', f'1500-{pdf['price'].max()}']  # Define the labels for the bins

# Create a new column with the binned data
pdf['price_bins'] = pd.cut(pdf['price'], bins=bins, labels=labels, right=True)

pdf = pdf.groupby(['brand', 'price_bins']).size().reset_index(name="counts")
px.bar(pdf.sort_values(['price_bins','counts']), x='price_bins', y='counts', color='brand')

/tmp/ipykernel_245564/1795029738.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



If we look closer at the distribution of the prices by brands we can see that the most amount of phones in cheap phones are again created by Xiaomi depicted in yellow color on the plot, and samsung, shown in pink color.

In [8]:
pdf = df
px.density_heatmap(pdf, x='release_year', y='brand', z='popularity_become_fan', histfunc='avg')

Some of the brands, like Nothing, have recieved massive popularity increase at the start and the popularity gradually decreased over time.
Some of the brands, like Samsung, have kept the number of likes on their phones consistent throughout the years.

The trend of phones recieving less likes over the years might be due to user engagement of GSMArena and might not be indicative.

In [9]:
pdf = df.groupby(['ip_rating'])['popularity_become_fan'].mean().reset_index()
px.bar(pdf.sort_values('popularity_become_fan', ascending=False), x='ip_rating', y='popularity_become_fan')

Phones with IP67 or IP68 water and dust protection recieve more likes on average than phones without any water and dust protection (IPXX).

Ultra-rugged indestructible phones with IP69 protection level do not recieve a lot of likes from people.

Apple recieves the most amount of views on average.

In [10]:
pdf = df.groupby(['release_year', '5g']).size().reset_index(name="counts")
px.bar(pdf, x='release_year', y='counts', color='5g')

Most of the phones adopted 5g technology.

In [11]:
px.scatter(df, x='price', y='popularity_become_fan', color='brand')

Samsung phones have recieved the most number of likes in mid price segment

In [12]:
pdf = df.groupby(['brand', 'internal_ram_gb']).size().reset_index(name="counts")
pdf['brand_total'] = pdf.groupby('brand')['counts'].transform('sum')
pdf['proportion_within_brand'] = pdf.apply(
    lambda row: row['counts'] / row['brand_total'] if row['brand_total'] > 0 else 0,
    axis=1
).round(3)

px.bar(pdf, x='brand', y='internal_ram_gb', color='proportion_within_brand', hover_data=['counts', 'proportion_within_brand'], # Show counts and proportion on hover
    labels={'proportion_within_brand': 'Proportion within Brand'}, # Update the color bar label
    title='Internal RAM Distribution (Color Relative to Brand Total)',
    barmode='group')

In [13]:
pdf = df.groupby(['screen_resolution_x', 'screen_resolution_y', 'foldable']).size().reset_index(name="counts")
px.scatter(pdf, x='screen_resolution_x', y='screen_resolution_y', size='counts', color='foldable', size_max=100)

Most of the phones have resolution of 1080x2400, which is 9:20 aspect ratio, followed by 720x1600 resolution, which is also 9:20 aspect ratio. There are several phones which have tablet-like resolution and aspect ratios, and this phones are foldable phones as can be seen from the plot

In [14]:
pdf = df
px.histogram(pdf, x='screen_size')

In [15]:
# pdf = df.groupby(['height_mm', 'length_mm', 'screen_size'])['popularity_become_fan'].mean().reset_index()
pdf = df
px.density_heatmap(pdf, x='screen_size',
                 y='brand', # Or perhaps 'width_mm' if that's the intended dimension
                 z='popularity_become_fan',
                 histfunc='avg')

Most prefered screen size lies between 6.5-7 inches.

Apple small phones with screen size of 4.5-5.5 inches receive the same popularity as their larger models.

Google smaller phones with screen size less than 6 inches also compete with the models that have larger screen.

In [16]:
pdf = df.groupby(['screen_hz'])['popularity_become_fan'].mean().reset_index()
px.bar(pdf.sort_values('popularity_become_fan'), x='screen_hz', y='popularity_become_fan')

Phones with 120 hz screen refresh rate are more popular than phones with basic refresh rate of 60 hz.

In [17]:
pdf = df.groupby(['screen_type', 'release_year'])['popularity_become_fan'].mean().reset_index()
px.scatter(pdf.sort_values('popularity_become_fan'), x='release_year', y='screen_type', size='popularity_become_fan', size_max=30)

LTPO OLED screen type is the most popular types of screen in a smartphone, followed by oled and amoled, while TN+Film screens are least popular.

IPS screens have plummeted in popularity over time. 

In [18]:
pdf = df
px.histogram(pdf, x='battery_capacity', nbins=10)

In [19]:
pdf = df.groupby('battery_capacity')['popularity_become_fan'].mean().reset_index()
px.histogram(pdf, x='battery_capacity', y='popularity_become_fan', nbins=40, histfunc='avg')

In [20]:
pdf = df.groupby('internal_rom_gb')['popularity_become_fan'].mean().reset_index()
pdf['internal_rom_gb'] = pdf['internal_rom_gb'].astype('str')
px.bar(pdf, x='internal_rom_gb', y='popularity_become_fan')

Phones with larger internal storage recieve more likes than phones with lower internal storage memory

In [21]:
pdf = df['camera_mp']
px.histogram(pdf)

In [22]:
pdf = df.groupby('camera_mp')['popularity_become_fan'].mean().reset_index()
px.histogram(pdf.sort_values('camera_mp'), x='camera_mp', y='popularity_become_fan', histfunc='avg', nbins=10)

In [23]:
pdf = df.groupby('camera_video_resolution')['popularity_become_fan'].mean().reset_index()
px.histogram(pdf, x='camera_video_resolution', y='popularity_become_fan', histfunc='avg', nbins=10)

In [24]:
pdf = df
px.density_heatmap(pdf, x='camera_video_resolution', y='camera_video_fps', z='popularity_become_fan', histfunc='avg')

Phones with maximum video resolution of 1080p do not recieve as musch popularity as phones with 4k and higher video resolution, for which the most popular framerate is 120 fps, with average likes on phones of 642.

Correlation analysis

In [25]:
df_corr = df.corr(numeric_only=True).round(1)['popularity_become_fan']
px.bar(df_corr.sort_values(ascending=False))

There is no strong correlation between phone characteristics and the popularity of a phone. The most correlated feature is the number of views a phone has recieved, which is expected. Screen resolution and usb version are the next most correlated metrics among phone characterestics.

In [26]:
df_corr = df[df['brand'] == 'Apple'].corr(numeric_only=True).round(1)['popularity_become_fan']
px.bar(df_corr.sort_values(ascending=False))

For Apple phones, the chipset architecture and the refresh rate of the screen correlates with higher popularity.